In [1]:
import os
os.chdir('../../..')

In [2]:
import convokit

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings

First we download the reddit corpus:

In [4]:
corpus = convokit.Corpus(filename="convokit/thread_generator/fake-corpus-trajectory-40")

In [5]:
corpus.print_summary_stats()

Number of Users: 200
Number of Utterances: 39960
Number of Conversations: 999


In [6]:
len(list(corpus.random_conversation().iter_utterances()))

40

In [7]:
# create a hyperconvo object and use it to extract features
hc = convokit.HyperConvo(prefix_len=20)
hc.fit_transform(corpus)

In [8]:
convo1 = corpus.random_conversation()
print(convo1.id)
convo1.meta['hyperconvo']

827-1


{'max[indegree over c->c responses]': 2,
 'argmax[indegree over c->c responses]': 11,
 'norm.max[indegree over c->c responses]': 0.10526315789473684,
 '2nd-largest[indegree over c->c responses]': 1,
 '2nd-argmax[indegree over c->c responses]': 0,
 'norm.2nd-largest[indegree over c->c responses]': 0.05263157894736842,
 'mean[indegree over c->c responses]': 0.95,
 'mean-nonzero[indegree over c->c responses]': 1.0555555555555556,
 'prop-nonzero[indegree over c->c responses]': 0.9,
 'prop-multiple[indegree over c->c responses]': 0.05555555555555555,
 'entropy[indegree over c->c responses]': 2.871476118054867,
 '2nd-largest / max[indegree over c->c responses]': 0.5,
 'max[outdegree over C->c responses]': 9,
 'max[indegree over C->c responses]': 2,
 'argmax[outdegree over C->c responses]': 0,
 'argmax[indegree over C->c responses]': 11,
 'norm.max[outdegree over C->c responses]': 0.47368421052631576,
 'norm.max[indegree over C->c responses]': 0.10526315789473684,
 '2nd-largest[outdegree over

In [9]:
from convokit import Classifier

In [10]:
doubled = 0
for convo in corpus.iter_conversations():
    utts40 = convo.get_chronological_utterance_list()
    utts20 = utts40[:20]
    num_users_20 = len(set(utt.user.id for utt in utts20))
    num_users_40 = len(set(utt.user.id for utt in utts40))
    convo.meta['doubled'] = (num_users_40 / num_users_20 >= 1.5)

In [11]:
corpus.random_conversation().meta['hyperconvo']

{'max[indegree over c->c responses]': 2,
 'argmax[indegree over c->c responses]': 17,
 'norm.max[indegree over c->c responses]': 0.10526315789473684,
 '2nd-largest[indegree over c->c responses]': 1,
 '2nd-argmax[indegree over c->c responses]': 0,
 'norm.2nd-largest[indegree over c->c responses]': 0.05263157894736842,
 'mean[indegree over c->c responses]': 0.95,
 'mean-nonzero[indegree over c->c responses]': 1.0555555555555556,
 'prop-nonzero[indegree over c->c responses]': 0.9,
 'prop-multiple[indegree over c->c responses]': 0.05555555555555555,
 'entropy[indegree over c->c responses]': 2.8714761180548676,
 '2nd-largest / max[indegree over c->c responses]': 0.5,
 'max[outdegree over C->c responses]': 9,
 'max[indegree over C->c responses]': 2,
 'argmax[outdegree over C->c responses]': 1,
 'argmax[indegree over C->c responses]': 17,
 'norm.max[outdegree over C->c responses]': 0.47368421052631576,
 'norm.max[indegree over C->c responses]': 0.10526315789473684,
 '2nd-largest[outdegree ove

In [12]:
for convo in corpus.iter_conversations():
    convo.meta.update(convo.meta['hyperconvo'])

In [13]:
hg_feats = list(corpus.random_conversation().meta['hyperconvo'])

In [25]:
clf = Classifier(obj_type="conversation", pred_feats=['hyperconvo'], labeller=lambda convo: convo.meta['doubled'])

In [28]:
clf.evaluate_with_train_test_split(corpus)

Using corpus objects...
Running a train-test-split evaluation...
Done.


(0.85, array([[107,  13],
        [ 17,  63]]))

In [ ]:
clf

In [15]:
corpus.random_conversation().meta['hyperconvo']

{'max[indegree over c->c responses]': 9,
 'argmax[indegree over c->c responses]': 0,
 'norm.max[indegree over c->c responses]': 0.47368421052631576,
 '2nd-largest[indegree over c->c responses]': 2,
 '2nd-argmax[indegree over c->c responses]': 1,
 'norm.2nd-largest[indegree over c->c responses]': 0.10526315789473684,
 'mean[indegree over c->c responses]': 0.95,
 'mean-nonzero[indegree over c->c responses]': 1.9,
 'prop-nonzero[indegree over c->c responses]': 0.5,
 'prop-multiple[indegree over c->c responses]': 0.2,
 'entropy[indegree over c->c responses]': 1.8306855287903425,
 '2nd-largest / max[indegree over c->c responses]': 0.2222222222222222,
 'max[outdegree over C->c responses]': 4,
 'max[indegree over C->c responses]': 9,
 'argmax[outdegree over C->c responses]': 11,
 'argmax[indegree over C->c responses]': 0,
 'norm.max[outdegree over C->c responses]': 0.21052631578947367,
 'norm.max[indegree over C->c responses]': 0.47368421052631576,
 '2nd-largest[outdegree over C->c responses]

In [16]:
from convokit.model import Corpus, Conversation, Utterance, User, CorpusObject
from typing import List, Union, Callable
import pandas as pd
from scipy.sparse import csr_matrix
import numpy as np
from convokit.model import warn
from scipy.sparse import vstack

In [17]:
from convokit import extract_feats_dict, extract_label_dict

In [18]:
obj_type = "conversation"
pred_feats = ["hyperconvo"]
selector=lambda convo: True
labeller = lambda convo: convo.meta['doubled']

In [19]:
obj_id_to_feats = extract_feats_dict(corpus, obj_type, pred_feats, selector)
obj_id_to_label = extract_label_dict(corpus, obj_type, labeller, selector)

X_df = pd.DataFrame.from_dict(obj_id_to_feats, orient='index')
y_df = pd.DataFrame.from_dict(obj_id_to_label, orient='index')

X_y_df = pd.concat([X_df, y_df], axis=1, sort=False)

y = X_y_df['y']
X = X_y_df.drop(columns='y')

In [20]:
X.astype('float64')

,max[indegree over c->c responses],argmax[indegree over c->c responses],norm.max[indegree over c->c responses],2nd-largest[indegree over c->c responses],2nd-argmax[indegree over c->c responses],norm.2nd-largest[indegree over c->c responses],mean[indegree over c->c responses],mean-nonzero[indegree over c->c responses],prop-nonzero[indegree over c->c responses],prop-multiple[indegree over c->c responses],...,is-present[reciprocity motif over mid-thread],count[reciprocity motif over mid-thread],is-present[external reciprocity motif over mid-thread],count[external reciprocity motif over mid-thread],is-present[dyadic interaction motif over mid-thread],count[dyadic interaction motif over mid-thread],is-present[incoming triads over mid-thread],count[incoming triads over mid-thread],is-present[outgoing triads over mid-thread],count[outgoing triads over mid-thread]
1-1,15.0,0.0,0.789474,1.0,1.0,0.052632,0.95,3.800000,0.25,0.200000,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0
10-1,14.0,0.0,0.736842,1.0,1.0,0.052632,0.95,3.166667,0.30,0.166667,...,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
100-1,17.0,0.0,0.894737,1.0,1.0,0.052632,0.95,6.333333,0.15,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101-1,13.0,0.0,0.684211,2.0,3.0,0.105263,0.95,3.166667,0.30,0.333333,...,1.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0,1.0,1.0
102-1,12.0,0.0,0.631579,1.0,2.0,0.052632,0.95,2.375000,0.40,0.125000,...,0.0,0.0,1.0,2.0,0.0,0.0,1.0,1.0,1.0,3.0
103-1,16.0,0.0,0.842105,1.0,1.0,0.052632,0.95,4.750000,0.20,0.250000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104-1,13.0,0.0,0.684211,2.0,3.0,0.105263,0.95,3.166667,0.30,0.333333,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,2.0,1.0,1.0
105-1,13.0,0.0,0.684211,2.0,1.0,0.105263,0.95,3.166667,0.30,0.333333,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,3.0,0.0,0.0
106-1,14.0,0.0,0.736842,1.0,1.0,0.052632,0.95,3.166667,0.30,0.166667,...,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
107-1,13.0,0.0,0.684211,2.0,7.0,0.105263,0.95,3.166667,0.30,0.333333,...,1.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0,1.0,1.0


In [21]:
X.values

array([[15, 0, 0.7894736842105263, ..., 0, True, 1],
       [14, 0, 0.7368421052631579, ..., 0, False, 0],
       [17, 0, 0.8947368421052632, ..., 0, False, 0],
       ...,
       [2, 1, 0.10526315789473684, ..., 2, True, 2],
       [2, 6, 0.10526315789473684, ..., 1, True, 1],
       [1, 0, 0.05263157894736842, ..., 0, False, 0]], dtype=object)

In [22]:
X.values

array([[15, 0, 0.7894736842105263, ..., 0, True, 1],
       [14, 0, 0.7368421052631579, ..., 0, False, 0],
       [17, 0, 0.8947368421052632, ..., 0, False, 0],
       ...,
       [2, 1, 0.10526315789473684, ..., 2, True, 2],
       [2, 6, 0.10526315789473684, ..., 1, True, 1],
       [1, 0, 0.05263157894736842, ..., 0, False, 0]], dtype=object)

In [23]:
csr_matrix(X.values), np.array(y)

TypeError: no supported conversion for types: (dtype('O'),)

In [24]:
clf.fit_transform(corpus)

Here are some examples of features computed over the three example threads from before:

In [ ]:
C_C_indegree_feats = [x for x in feat_names if 'indegree over C->C responses' in x]
C_C_mid_outdegree_feats = [x for x in feat_names if 'outdegree over C->C mid-thread responses' in x]
motif_count_feats = [x for x in feat_names if ('count' in x) and ('mid' not in x)]

Features derived from the distribution of hypernode to hypernode indegrees (i.e., "how many other people are responding to a particular discussion participant?"). As in the paper, we compute various summary statistics over the distribution. 

As noted at the start of this notebook, since we do not have access to reaction information, these distributions encompass only the reply structure within the thread.

In [ ]:
feat_df.loc[demo_threads][C_C_indegree_feats].T.sort_index()

Features derived from the distribution of hypernode to hypernode outdegrees in the _middle_ of the thread (i.e., "beyond the root comment, how many other people is each participant responding to?"). 

In [ ]:
feat_df.loc[demo_threads][C_C_mid_outdegree_feats].T.sort_index()

Features derived from the 5 motifs considered in the paper. Note that because we do not have reply information, we instead take as features _counts_ of each motif (in the paper, we would additionally compare reply or reaction edge types within motifs of a particular form)

In [ ]:
feat_df.loc[demo_threads][motif_count_feats].T.sort_index()

While the features generated above can be used as is (e.g., as features in a prediction task), we can also interpret them by projecting them into a low-dimensional space. 

In [ ]:
from sklearn.preprocessing import Imputer, Normalizer, StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import pairwise_distances

In [ ]:
feat_mtx = Imputer(axis=1, missing_values=-1).fit_transform(feat_df.values)
feat_mtx = StandardScaler().fit_transform(feat_mtx)

In [ ]:
svd = TruncatedSVD(n_components=7, algorithm='arpack') # deals with an issue where the randomized alg hangs
svd.fit(feat_mtx)
U, s, V = svd.transform(feat_mtx) / svd.singular_values_, \
        svd.singular_values_, \
        svd.components_.T

In [ ]:
U_norm = Normalizer().fit_transform(U)
V_norm = Normalizer().fit_transform(V)
U_df = pd.DataFrame(data=U_norm, index=feat_df.index)
V_df = pd.DataFrame(data=V_norm, index=feat_names)

This embedding procedure produces embeddings of threads in the low-dimensional space:

In [ ]:
U_df.loc[demo_threads].T

As well as embeddings of features:

In [ ]:
V_df.loc[C_C_indegree_feats]

In [ ]:
convo1.meta

As in the paper, for further interpretability we can consider embeddings of _communities_ (subreddits, standing for Facebook pages) in terms of the discussions they foster, by averaging the embeddings of all threads in a particular subreddit.

In [ ]:
subreddits = [convo.meta['original_convo_meta']['subreddit'] for convo in threads_corpus.iter_conversations()]

In [ ]:
len(subreddits)

In [ ]:
U_df['subreddit'] = subreddits

In [ ]:
subreddit_means = U_df.groupby('subreddit').mean()
subreddit_df = pd.DataFrame(
        data=Normalizer().fit_transform(subreddit_means.values),
        index = subreddit_means.index
    )

First, let's give a rough overview of the space we've sketched out through this procedure, by visualizing the subreddit embeddings using the TSNE algorithm.

In [ ]:
tsne = TSNE(random_state=2018)
tsne_df = pd.DataFrame(data=tsne.fit_transform(subreddit_df.values),
                      index=subreddit_df.index)

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(tsne_df[0].values, tsne_df[1].values)
for i, txt in enumerate(tsne_df.index):
    plt.annotate(txt, (tsne_df.values[i,0], tsne_df.values[i,1]))
plt.show()

Looking across this plot, we can spot a few interpretable-looking groupings:

In [ ]:
games = ["battlefield3", "Diablo", "DotA2", "Guildwars2", "leagueoflegends", "magicTCG", "Minecraft", "pokemon", "skyrim", "starcraft", "tf2", "wow"]
trade = ["Dota2Trade", "pokemontrades", "SteamGameSwap", "tf2trade", "Random_Acts_Of_Amazon"]
sports = ["baseball", "CFB", "hockey", "MMA", "nba", "nfl", "soccer"]

link_aggregators = ["AskReddit", "WTF", "pics", "gifs", "aww", "funny", "todayilearned",
                   "AdviceAnimals"]
relationships = ["AskMen", "AskWomen", "relationships", "relationship_advice", "OkCupid"]

plt.scatter(tsne_df[0].values, tsne_df[1].values, color="#dddddd")
plt.scatter(tsne_df[0].values, tsne_df[1].values, color=[
    "green" if l in games else
    "gold" if l in trade else
    "purple" if l in relationships else
    "red" if l in link_aggregators else
    "blue" if l in sports else
    "#00000000"
    for l in tsne_df.index])
plt.show()

For instance, video games (e.g., tf2, DotA2; red) tend to group together, along with buy-sell subreddits (e.g., tf2trade, Dota2Trade, yellow), subreddits related to relationships (e.g., AskWomen, relationship_advice; green) and large default-subreddit hubs for sharing random links (e.g., pics, AskReddit; purple).

There may be other interesting groupings that you may spot as well -- for instance, a vaguely right-wing MensRights cluster (with Libertarian, guns); a rather intriguing cluster consisting of politics, sex, business, etc.

Some topical groups are more diffuse -- for instance, sports-based subreddits (blue). Perhaps this is a limitation of our representation, or that these subreddits actually foster very different interactional dynamics.

Another way to delve into these groupings is to look at nearest neighbors of subreddits, in terms of the embedding:

In [ ]:
dists = pairwise_distances(subreddit_df.values, metric='cosine')
flat_dists = np.ravel(dists)
idx1, idx2 = np.unravel_index(np.arange(len(flat_dists)), dists.shape)
pairwise_dist_df = pd.DataFrame.from_dict({'p1': subreddit_df.index[idx1],
                                           'p2': subreddit_df.index[idx2],
                                           'dist': flat_dists},
                                           orient='columns')

In [ ]:
def print_nearest_neighbors(pairwise_dist_df, 
                           test_subreddits=[],
                           top_N=10):
    for subreddit in test_subreddits:
        subset_df = pairwise_dist_df[(pairwise_dist_df.p1 == subreddit)
                                 & (pairwise_dist_df.p2 != subreddit)]
        print(subreddit)
        print(subset_df.sort_values('dist')[['p2', 'dist']].head(top_N))
        print()

In [ ]:
print_nearest_neighbors(pairwise_dist_df, ['apple', 'politics', 'leagueoflegends',
                                          'AskWomen', 'Music', 'pics',
                                          'australia', 'Random_Acts_Of_Amazon',
                                          'Bitcoin', 'MensRights'])

We can also try to interpret each dimension of the embedding -- roughly speaking the threads, features and subreddits with extremal values along one dimension could be seen as characterizing a particular "type" of discussion, in terms of the discussion structure.

In [ ]:
from IPython.display import display

In [ ]:
def display_dimension(dim, ascending=True, n=5):
    top_threads = U_df.sort_values(dim, ascending=ascending).head(n)
    display(top_threads)
    display(V_df.sort_values(dim, ascending=ascending).head(n))
    display(subreddit_df.sort_values(dim, ascending=ascending).head(n))
    return top_threads.index

For instance, the first latent dimension divides the space of Reddit discussions between focused dialogues involving 2 people who repeatedly interact, and "expansionary" threads involving multiple people who generally only engage once (as with the corresponding dimension discussed in the paper, this echoes the contrast explored in papers such as Backstrom et. al, 2013). At the subreddit level, we see a divide between subreddits that are selling things (perhaps the dialogues consist of a buyer and a seller) and large default link-sharing subreddits like AskReddit and pics.

In [ ]:
top_threads = display_dimension(0, n=10)